In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## 读入数据

In [2]:
def loadDataSet(filename):
    frTrain = open(filename)
    featureSet = []
    labels = []
    for line in frTrain.readlines():
        currLine = line.strip().split('\t')
        lineArr = []
        for i in range(21):
            lineArr.append(float(currLine[i]))
        featureSet.append(lineArr)
        labels.append(float(currLine[21]))
    return featureSet, labels

## Sigmoid 函数

In [3]:
def sigmoid(inX):
    return 1.0/(1+np.exp(-inX))

## 梯度上升优化算法
**梯度上升伪代码：**  
每个回归系数初始化为1  
重复R次：  
&ensp;&ensp;&ensp;&ensp;计算整个数据集的梯度  
&ensp;&ensp;&ensp;&ensp;使用 alpha\*gradient 更新回归系数的向量  
返回回归系数

In [4]:
def gradAscent(dataMatIn, classLabels):
    # 将数据转换成 numpy 矩阵
    dataMatrix = np.mat(dataMatIn)    # 100*3
    labelMat = np.mat(classLabels).transpose()    # 1*100 --> 100*1
    m, n = np.shape(dataMatrix)
    alpha = 0.001    # 步长
    maxCycles = 500    # 迭代次数
    weights = np.ones((n, 1))    # n*1
    for k in range(maxCycles):
        h = sigmoid(dataMatrix * weights)    # (100*3)*(3*1) --> 100*1
        error = (labelMat - h)    # 100*1
        weights = weights + alpha * dataMatrix.transpose() * error
    return weights

## 随机梯度上升算法
随机梯度上升算法：一次仅用一个样本点来更新回归系数  
**随机梯度上升伪代码：**  
每个回归系数初始化为1  
对数据集中每个样本：  
&ensp;&ensp;&ensp;&ensp;计算该样本的梯度  
&ensp;&ensp;&ensp;&ensp;使用 alpha\*gradient 更新回归系数的向量  
返回回归系数

In [5]:
def stocGradAscent0(dataMatrix, classLabels):
    m, n = np.shape(dataMatrix)
    alpha = 0.01
    weights = np.ones(n)    # 1*n
    for i in range(m):
        h = sigmoid(sum(dataMatrix[i]*weights))
        error = classLabels[i] - h
        weights = weights + alpha * error * dataMatrix[i]
    return weights

### 改进的随机梯度上升算法

In [6]:
def stocGradAscent1(dataMatrix, classLabels, numIter=150):
    m, n = np.shape(dataMatrix)
    weights = np.ones(n)    # 1*n
    for j in range(numIter):
        dataIndex = list(range(m))
        for i in range(m):
            alpha = 4 / (1.0 + j + i) + 0.05    # 每次迭代调整alpha
            randIndex = int(np.random.uniform(0, len(dataIndex)))    # 随机选取样本   
            h = sigmoid(sum(dataMatrix[randIndex]*weights))
            error = classLabels[randIndex] - h
            weights = weights + alpha * error * dataMatrix[randIndex]
            del(dataIndex[randIndex])
    return weights

## 准备数据
处理特征缺失值常用方法
- 使用可用特征的均值来补缺失值
- 使用特殊值填补，如-1
- 忽略有缺失值的样本
- 使用相似样本的均值来补缺失值
- 使用另外的机器学习算法预测缺失值  
处理类别标签缺失值方法： 丢弃该条样本  
  
本节中根据逻辑回归的特点，将缺失值用0替代，既可保留数据，也不影响优化算法
此节使用已处理好的数据

## Logistic回归分类函数

In [7]:
def classifyVector(inX, weights):
    prob = sigmoid(sum(inX * weights))
    if prob > 0.5:
        return 1.0
    else:
        return 0.0

In [8]:
def colicTest():
    # 读入训练数据
    trainingSet,trainingLabels = loadDataSet('horseColicTraining.txt')
    # 使用随机梯度下降进行系数学习
    trainWeights = stocGradAscent1(np.array(trainingSet), trainingLabels, 666)
    errorCount = 0
    # 读入测试数据
    testSet,testLabels = loadDataSet('horseColicTest.txt')
    for i in range(len(testLabels)):
        if classifyVector(testSet[i], trainWeights) != testLabels[i]:
            errorCount += 1
    # 计算错误率
    errorRate = float(errorCount)/float(len(testLabels))
    print('the error rate of this test is: %f' % errorRate)
    return errorRate

def multiTest():
    numTests = 10
    errorSum = 0.0
    for k in range(numTests):
        errorSum += colicTest()
    print('after %d iterations the average error rate is: %f' % (numTests, errorSum/float(numTests)))

In [9]:
multiTest()

the error rate of this test is: 0.343284
the error rate of this test is: 0.313433
the error rate of this test is: 0.328358
the error rate of this test is: 0.268657
the error rate of this test is: 0.388060
the error rate of this test is: 0.343284
the error rate of this test is: 0.447761
the error rate of this test is: 0.447761
the error rate of this test is: 0.447761
the error rate of this test is: 0.358209
after 10 iterations the average error rate is: 0.368657
